Note: PyTorch模型训练实用教程(余霆嵩)
----
<https://github.com/tensor-yu/PyTorch_Tutorial>

**1.3 让pytorch能读你的数据-1 制作图片的索引**

In [1]:
import os
'''
    为数据集生成对应的txt文件
'''

'\n    为数据集生成对应的txt文件\n'

In [2]:
train_txt_path = os.path.join("Data", "train.txt")
train_dir = os.path.join("Data", "train")
valid_txt_path = os.path.join("Data", "valid.txt")
valid_dir = os.path.join("Data", "valid")

In [5]:
def gen_txt(txt_path, img_dir):
    f = open(txt_path,'w')
    
    for root, s_dirs, _ in os.walk(img_dir, topdown=True): #获取train文件夹下的各文件夹名称
        for sub_dir in s_dirs: # 进入到每一个文件夹中，共10个
            i_dir = os.path.join(root,sub_dir)
            img_list = os.listdir(i_dir)
            for i in range(len(img_list)):
                if not img_list[i].endswith('png'): # 若不是png文件，跳过
                    continue
                label = img_list[i].split('_')[0]
                img_path = os.path.join(i_dir, img_list[i])
                line = img_path + ' '+label+ '\n'
                f.write(line)
    f.close()

if __name__ =='__main__':
    gen_txt(train_txt_path, train_dir)
    gen_txt(valid_txt_path, valid_dir)
            

**1.3 让pytorch能读你的数据-2 构建Dataset子类**

In [6]:
from PIL import Image
from torch.utils.data import Dataset

In [7]:
class MyDataset(Dataset):
    def __init__(self, txt_path, transform= None, target_transform=None):
        fh = open(txt_path, 'r')
        imgs = []
        for line in fh:
            line = line.rstrip()
            words = line.split()
            imgs.append((words[0], int(words[1])))
        self.imgs = imgs # 最主要就是生成这个List, 然后DataLoader中给index, 然后通过getitem读取图片数据
        self.transform = transform
        self.target_transform = target_transform
        
    def __getitem__(self, index):
        fn, label = self.imgs[index]
        img = Image.open(fn).convert('RGB') # 像素值0~255， 在transform.totensor会除以255， 使像素值变成0~1
        
        if self.transform is not None:
            img = self.transform(img) #在这里做transform, 转为tensor等等
        return img, label
    
    def __len__(self):
        return len(self.imgs)       
        